<a href="https://colab.research.google.com/github/Elizabeth14755/DSL/blob/main/Job3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token: (class, value)
class: int
value: str
var: str                 # name of the non-terminal symbol
definition: list(rule)
rule: list(var | token)  # right part of the rule
```

In [ ]:
def get_vanish(grammar) -> set:
  """
  Search for vanish symbols.

  :param grammar: free grammar without left recursion
  :return: set of vanishing nonterminals symbols
  """
  def all_vanish_symbols(rule: str) -> bool:
    for s in rule:
      if s not in vanish and s != (0, ''):
        return False
    return True    
  
  vanish = set()

  # on each step add all nonterminal symbols for which there is a rule that contains only vanishing vars or empty token
  prev_step_number = -1
  while len(vanish) != prev_step_number:
    prev_step_number = len(vanish)
    vanish.update([v for v, defin in grammar['vars'].items() if list(filter(all_vanish_symbols, defin)) != []])

  return vanish  

In [1]:
def FIRST(grammar, s: str) -> set:
  """
    Find the set of terminal symbols that strings derived from A begin with. ({a: A ->> aα})

   :param grammar: free grammar without left recursion
   :param s: nonterminal symbol of the grammar
   :return: set of possible terminals
            if the given nonterminal symbol is vanish, then the end of input is added to the returned set
  """
  terminals = set()

  for definition in grammar['vars'][s]:
    index = 0
    # add FIRST of each vanish symbol of the begging of the rule definition
    while index < len(definition) and definition[index] in get_vanish(grammar):
      terminals.update(FIRST(grammar, definition[index]))
      index += 1
    if index < len(definition) and definition[index] not in grammar['toks']:
      # add the FIRST of the first not vanish nonterminal symbol of the rule definition
      terminals.update(FIRST(grammar, definition[index]))
    elif index < len(definition):
      # add the first terminal symbol of the rule definition
      terminals.add(definition[index])

  if s in get_vanish(grammar):
    terminals.add((7, '⊳'))
    
  return terminals

In [2]:
def FOLLOW(grammar, s: str) -> set:
  """
   Find the set of terminal symbols that can follow after the given nonterminal symbol A. ({a: S ->> αAaβ})

   :param grammar: free grammar without left recursion
   :param s: nonterminal symbol of the grammar
   :return: set of possible terminals
            if S ->> αA, then the end of input is added to the returned set
  """
  terminals = set()

  for var, definitions in grammar['vars'].items():
    for definition in definitions:
      for i in [i for i,x in enumerate(definition) if x == s]:
        # add FIRST of each vanish symbol that follows after the given nonterminal symbol
        j = 1
        next_symbol = definition[i + j] if j < len(definition) else None
        while next_symbol and next_symbol in get_vanish(grammar):
          terminals.update(first(grammar, next_symbol))
          j += 1

        # add FOLLOW of var if the rule definition ends with the given nonterminal symbol
        if var != s and j == len(definition):
          terminals.update(follow(grammar, var))  

        # add first terminal symbol that follows after the given nonterminal symbol
        if j < len(definition) and definition[j] in grammar['toks']:
          terminals.add(definition[j])
        # add the FIRST of the first not vanish nonterminal symbol of the rule definition
        elif j < len(definition):
          terminals.update(first(grammar, definition[j]))
        else:
          terminals.add((7, '⊳'))

  return terminals

In [ ]:
"""
  S  -> TS'
  S' -> +TS' ∣ e
  T  -> FT'
  T' -> *FT' ∣ e
  F  -> v ∣ n ∣ (S)
"""
grammar = {
    'toks': {(0, ''), (1, '+'), (2, '*'), (3, 'v'), (4, 'n'), (5, '('), (6, ')'), (7, '⊳')},
    'vars': {
        'S': [['T', "S'"]], 
        "S'": [[(1, '+'), 'T', "S'"], 
                [(0, '')]], 
        'T': [['F', "T'"]],
        "T'": [[(2, '*'), 'F', "T'"],
                [(0, '')]],
        'F': [[(3, 'v')],
              [(4, 'n')],
              [(5, '('), 'S', (6, ')')]]
        },
    'hvar': 'S'
    }

print("FIRST(S) = {}".format(FIRST(grammar, "S")))
print("FIRST(T) = {}".format(FIRST(grammar, "T")))
print("FIRST(F) = {}".format(FIRST(grammar, "F")))
print("FIRST(S') = {}".format(FIRST(grammar, "S'")))
print("FIRST(T') = {}".format(FIRST(grammar, "T'")))

print("-----------------------------------------------")

print("FOLLOW(S) = {}".format(FIRST(grammar, "S")))
print("FOLLOW(T) = {}".format(FIRST(grammar, "T")))
print("FOLLOW(F) = {}".format(FIRST(grammar, "F")))
print("FOLLOW(S') = {}".format(FIRST(grammar, "S'")))
print("FOLLOW(T') = {}".format(FIRST(grammar, "T'")))

FIRST(S) = {(5, '('), (3, 'v'), (4, 'n')}
FIRST(T) = {(5, '('), (3, 'v'), (4, 'n')}
FIRST(F) = {(5, '('), (3, 'v'), (4, 'n')}
FIRST(S') = {(7, '⊳'), (0, ''), (1, '+')}
FIRST(T') = {(2, '*'), (0, ''), (7, '⊳')}
-----------------------------------------------
FOLLOW(S) = {(5, '('), (3, 'v'), (4, 'n')}
FOLLOW(T) = {(5, '('), (3, 'v'), (4, 'n')}
FOLLOW(F) = {(5, '('), (3, 'v'), (4, 'n')}
FOLLOW(S') = {(7, '⊳'), (0, ''), (1, '+')}
FOLLOW(T') = {(2, '*'), (0, ''), (7, '⊳')}
